<a href="https://colab.research.google.com/github/Zihooo/Auto-Scale-Development/blob/main/Study_1_Scale_Development_Example_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Item generation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Load pacakges
!pip install openai
import pandas as pd

In [ ]:
import openai
import os

# insert your API here
os.environ["OPENAI_API_KEY"] = "Your API key"
openai.api_key = 'Your API key'
# Upload training data
from openai import OpenAI

client = OpenAI()

In [ ]:
# users need to change the construct definition.
prompt_IDT = [{"role": "system", "content": """You are ChatGPT, a large language model trained by OpenAI. You will use the following instructions to craft your response:

1. **Task Overview**
   - You need to create unique Likert-type measurement items related to a concept called *Interpersonal Distrust*.
   - Interpersonal Distrust is An expectation of harmful, hostile, or other negative outcomes in interactions with another person, accompanied by negative affect and an intention to take protective actions based on these expectations.

2. **Dimensions of Interpersonal Distrust**
   - **Cognitive**: Rational beliefs or expectations about another party’s untrustworthiness, specifying in what respects and under which circumstances distrust may arise.
   - **Behavioral**: The unwillingness or avoidance of future interactions or risk-taking actions based on these negative expectations and feelings toward the other person.
   - **Affective**: Negative emotions directed at the distrusted person.

3. **Specific Instructions**
   - Each item belonging to one of the three dimensions above (*Awareness*, *Attention*, or *Acceptance*).
   - Each statement should relate clearly to its assigned dimension.
   - Provide each statement on a new line.
   - All statement should be positively keyed.
   - Do not strictly replicate the examples given; it is sufficient that each statement logically pertains to its dimension.
   - Items should be in Likert-type format.
   - Items should use short and simple language.
   - Items should correctly measure the target construct.
   - Items should not be double-barreled.
   - Items should not be such that virtually everyone or no one will endorse them.
   - Items should avoid colloquialisms that may not be familiar across age, ethnicity, region, gender, and so forth.
   - Items should avoid vague words such as many, most, often, or sometimes.
   - Items should be understandable to anyone who has graduated from high school.

4. **Important Requirements**
   - Avoid generate highly similar statements!!!Use different words. Avoid similar sentence structures.
   - Generate exactly 40 item for each dimension.

5. **Formatting Requirements**
   - Create two columns:
     1. The first column contains the statement.
     2. The second column identifies the *target dimension* of that statement (i.e., *Cognitive*, *Behavioral*, or *Affective*).
   - Each statement must be unique.

6. **Examples (Do Not Copy Exactly)**
   - Cognitive: “This person would behave in a deceptive and fraudulent way.”,"I am suspicious of the way this person will act in the future.", "This person would use me for his/her own benefits."
   - Behavioral: “I find it necessary to be cautious with this person.”,"I will protect myself from being taken advantage of by this person.","I will not count on this person for important things."
   - Affective: “I feel tense when I am with this person.”,"I experience anxiety when interacting with this person.","I worry about future interactions with this person."

7. **Final Output**
   - Return only the list of statements with their corresponding dimensions in two columns.
   - No additional commentary or explanation is needed.

Follow these instructions precisely! and produce your final answer accordingly.
"""},
      {"role": "user", "content": "Create exactly 40 items for each dimension in Interpersonal Distrust. Only provide the items, start a new line for each item."
      }]

In [ ]:
completion = client.chat.completions.create(
  model='gpt-4o-mini',
  messages=prompt,
  temperature=1.0,
  top_p = 0.80,
  frequency_penalty = 0.40,
  presence_penalty = 0.55
)

In [ ]:
completion.choices[0].message.content

"I expect this person to hide important information., Cognitive  \nI think this person may be deceitful., Cognitive  \nI am convinced that this person is not reliable., Cognitive  \nI believe this person is likely to have an ulterior motive., Cognitive  \nI suspect that this person may act in a manipulative way., Cognitive  \nI suspect that this person is driven by self-interest., Cognitive  \nI think this person may take advantage of others., Cognitive  \nI believe this person hides their true intentions., Cognitive  \nI expect that this person will act against my interests., Cognitive  \nI am convinced that this person prioritizes personal gain over fairness., Cognitive  \nI believe this person might exploit my weaknesses., Cognitive  \nI suspect that this person keeps secrets to harm others., Cognitive  \nI think this person is likely to twist facts., Cognitive  \nI expect that this person will not act with transparency., Cognitive  \nI am convinced that this person has a hidden age

In [ ]:
texts = completion.choices[0].message.content.split('\n')

In [ ]:
# one issue here is that GPT did not generate exactly 40 items for each dimension, which was stated in the prompt (I accidently deleted the output)
# in my previous output, 37 items were generated for each dimension. I manually generated 3 more to make it 40 for each dimension.
texts

In [ ]:
#optional: save generated items to a csv file
# Create a list to hold the processed rows
rows = []

for entry in texts:
    entry = entry.strip()
    if "," in entry:
        # Try splitting on the last comma
        parts = entry.rsplit(",", 1)
        if len(parts) == 2:
            text, label = parts[0].strip(), parts[1].strip()
        else:
            # Fallback if for some reason we don't get two parts
            text, label = entry, ""
    else:
        # Decide how to handle entries without a comma
        # Here we assign the whole entry as text and leave label empty
        text, label = entry, ""
        print(f"Warning: No comma found in entry: '{entry}'")

    rows.append({"Text": text, "Label": label})

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(rows)

# Save the DataFrame to a CSV file (without the index)
#df.to_csv("output.csv", index=False)

# Step 2: remove redundant items

In [ ]:
# I did this step on a seperate file. So I just put the example codes here.

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model_raw = SentenceTransformer("all-MiniLM-L6-v2") # Replace with your actual model path


In [ ]:
# use cognitive dimension as an example
sentences = [
    "I expect this person to hide important information.",
    "I think this person may be deceitful.",
    "I am convinced that this person is not reliable.",
    "I believe this person is likely to have an ulterior motive.",
    "I suspect that this person may act in a manipulative way.",
    "I suspect that this person is driven by self-interest.",
    "I think this person may take advantage of others.",
    "I believe this person hides their true intentions.",
    "I expect that this person will act against my interests.",
    "I am convinced that this person prioritizes personal gain over fairness.",
    "I believe this person might exploit my weaknesses.",
    "I suspect that this person keeps secrets to harm others.",
    "I think this person is likely to twist facts.",
    "I expect that this person will not act with transparency.",
    "I am convinced that this person has a hidden agenda.",
    "I believe this person is not likely to be straightforward.",
    "I suspect that this person may conceal their true aims.",
    "I think this person will choose self-interest over loyalty.",
    "I believe this person might deceive for personal benefit.",
    "I expect this person to be guarded about their intentions.",
    "I suspect this person would use others to fulfill their needs.",
    "I think this person values their own benefit over honesty.",
    "I believe this person may mislead others intentionally.",
    "I expect this person to engage in untrustworthy behavior.",
    "I suspect that this person has a tendency toward dishonesty.",
    "I think this person prizes personal gain more than truth.",
    "I believe this person acts with a concealed motive.",
    "I expect this person to put self-interest ahead of others.",
    "I suspect that this person may not honor their promises.",
    "I believe this person harbors secret judgments about others.",
    "I suspect that this person is inclined to pursue personal advantage.",
    "I think this person will likely act in a self-serving way.",
    "I expect that this person will disregard fairness for personal gain.",
    "I suspect that this person is predisposed to withhold trust.",
    "I believe this person chooses manipulation over sincerity.",
    "I think this person makes decisions with hidden advantages in mind.",
    "I expect that this person will be unpredictable in honest matters.",
    "I suspect that this person may twist facts for their benefit.",
    "I believe this person is not capable of full honesty.",
    "I think this person’s actions prove a lack of genuine concern for others."]

In [ ]:
embeddings_raw = model_raw.encode(sentences, convert_to_numpy=True)
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
similarity_matrix_raw = cosine_similarity(embeddings_raw)

In [ ]:
# iteratively removing the first item in the pair if cosine similarity is larger than 0.8
def drop_highly_similar(embeddings, thresh=0.8, verbose=False):
    """
    Iteratively drop the first vector in any pair whose cosine similarity
    exceeds `thresh`. Stops when all remaining pairs are ≤ `thresh`.

    Parameters
    ----------
    embeddings : (n, dim) array-like
        The embedding vectors.
    thresh : float, optional (default = 0.8)
        Similarity cut-off.
    verbose : bool, optional
        If True, prints which indices are removed at each iteration.

    Returns
    -------
    np.ndarray
        Filtered embeddings (order preserved for kept items).
    list[int]
        Original indices of the kept items.
    """
    emb = np.asarray(embeddings)
    keep = np.arange(len(emb))           # original indices still alive

    while True:
        sims = cosine_similarity(emb[keep])   # (m × m) matrix
        sims[np.triu_indices_from(sims)] = 0  # zero out self + upper tri → look once

        # find a pair (i,j) with similarity > thresh
        i, j = np.where(sims > thresh)
        if len(i) == 0:                      # nothing left to drop
            break

        first_to_drop = keep[np.min(i)]      # the lower index (first item)
        if verbose:
            print(f"Dropping original row {first_to_drop} "
                  f"(similarity {sims[i[0], j[0]]:.3f} with row {keep[j[0]]})")
        keep = keep[keep != first_to_drop]   # remove it and loop again

    return embeddings[keep], keep.tolist()

In [ ]:
filtered_emb, kept_indices = drop_highly_similar(embeddings_raw,
                          thresh=0.8,
                          verbose=True)
# pull out the surviving items -------------------------------
filtered_items = [sentences[i] for i in kept_indices]   # texts is your original list
print(filtered_items)                               # or do whatever you need

#Content Validation

##GPT

In [ ]:
items_cognitive = [
    "I expect this person to hide important information.",
    "I think this person may be deceitful.",
    "I am convinced that this person is not reliable.",
    "I believe this person is likely to have an ulterior motive.",
    "I suspect that this person may act in a manipulative way.",
    "I suspect that this person is driven by self-interest.",
    "I think this person may take advantage of others.",
    "I believe this person hides their true intentions.",
    "I expect that this person will act against my interests.",
    "I am convinced that this person prioritizes personal gain over fairness.",
    "I believe this person might exploit my weaknesses.",
    "I suspect that this person keeps secrets to harm others.",
    "I think this person is likely to twist facts.",
    "I expect that this person will not act with transparency.",
    "I am convinced that this person has a hidden agenda.",
    "I believe this person is not likely to be straightforward.",
    "I suspect that this person may conceal their true aims.",
    "I think this person will choose self-interest over loyalty.",
    "I believe this person might deceive for personal benefit.",
    "I expect this person to be guarded about their intentions.",
    "I suspect this person would use others to fulfill their needs.",
    "I think this person values their own benefit over honesty.",
    "I believe this person may mislead others intentionally.",
    "I expect this person to engage in untrustworthy behavior.",
    "I suspect that this person has a tendency toward dishonesty.",
    "I think this person prizes personal gain more than truth.",
    "I believe this person acts with a concealed motive.",
    "I expect this person to put self-interest ahead of others.",
    "I suspect that this person may not honor their promises.",
    "I believe this person harbors secret judgments about others.",
    "I suspect that this person is inclined to pursue personal advantage.",
    "I think this person will likely act in a self-serving way.",
    "I expect that this person will disregard fairness for personal gain.",
    "I suspect that this person is predisposed to withhold trust.",
    "I believe this person chooses manipulation over sincerity.",
    "I think this person makes decisions with hidden advantages in mind.",
    "I expect that this person will be unpredictable in honest matters.",
    "I suspect that this person may twist facts for their benefit.",
    "I believe this person is not capable of full honesty.",
    "I think this person’s actions prove a lack of genuine concern for others."
]

In [ ]:
items_behavioral = [
    "I avoid sharing personal details with this person.",
    "I keep my distance from this person in important matters.",
    "I choose not to rely on this person for help.",
    "I cancel plans with this person when possible.",
    "I refrain from involving this person in confidential issues.",
    "I guard my secrets when dealing with this person.",
    "I am careful about letting this person into my personal life.",
    "I limit my interactions with this person.",
    "I prefer to keep communication with this person brief.",
    "I protect myself by avoiding deep connections with this person.",
    "I hesitate to trust this person in important decisions.",
    "I resist sharing my true feelings with this person.",
    "I steer away from relying on this person in risky situations.",
    "I choose to monitor my actions when I am around this person.",
    "I take steps to avoid potential harm from this person.",
    "I withhold support from this person when it might put me at risk.",
    "I decide against disclosing personal struggles to this person.",
    "I remain cautious when inviting this person into my circle.",
    "I prefer to handle challenges on my own with this person present.",
    "I limit plans that involve future interactions with this person.",
    "I refuse to depend on this person for critical tasks.",
    "I actively reduce the time I spend with this person.",
    "I take care not to expose vulnerabilities near this person.",
    "I choose safe options when this person is involved.",
    "I avoid risky situations that include this person.",
    "I keep my guard up when interacting with this person.",
    "I refrain from sharing extra details around this person.",
    "I show reluctance to work jointly with this person.",
    "I prefer independent routines over teamwork with this person.",
    "I suppress my natural openness when I am with this person.",
    "I opt for minimal engagement when this person is present.",
    "I hide personal opinions to protect myself around this person.",
    "I keep my personal projects private from this person.",
    "I carefully choose what to share when this person is involved.",
    "I diminish my collaboration with this person when it is possible.",
    "I favor solitary actions over joint efforts with this person.",
    "I plan alternative routes to avoid activities with this person.",
    "I choose self-reliance when this person is part of a task.",
    "I opt out of group events that involve this person.",
    "I isolate myself from opportunities that include this person."
]

In [ ]:
items_affective = [
    "I feel uneasy when interacting with this person.",
    "I experience discomfort around this person.",
    "I feel alarmed in the presence of this person.",
    "I experience tension during our conversations.",
    "I feel anxious at the thought of meeting this person.",
    "I feel a constant sense of unease around this person.",
    "I sense a deep worry when I am with this person.",
    "I feel nervous about this persons actions.",
    "I feel stress when I think about this person.",
    "I experience a strong tension near this person.",
    "I feel discomfort during my encounters with this person.",
    "I become apprehensive when I see this person.",
    "I feel nervous during conversations with this person.",
    "I sense anxiety when I consider this person.",
    "I feel a surge of apprehension when this person is near.",
    "I experience unease when thinking about future meetings with this person.",
    "I feel troubled by this persons presence.",
    "I feel a strong worry when considering this person.",
    "I feel unsettled when interacting with this person.",
    "I sense a form of dread around this person.",
    "I feel restless when I am around this person.",
    "I experience a heightened state of alertness near this person.",
    "I feel a surge of anxiety in this persons company.",
    "I feel uneasy when I learn about this persons plans.",
    "I sense a nervous energy when facing this person.",
    "I feel disquiet when I am close to this person.",
    "I experience distress when this person is present.",
    "I feel a spike of nervousness during encounters with this person.",
    "I sense a looming fear in my interactions with this person.",
    "I feel jittery around this person.",
    "I experience a quick surge of fear when I am with this person.",
    "I feel a load of anxiety when expecting to meet this person.",
    "I feel unsettled before meeting this person.",
    "I experience distress when I consider being around this person.",
    "I feel my heart race in the presence of this person.",
    "I feel a consistent sense of worry when near this person.",
    "I experience internal conflict when interacting with this person.",
    "I feel a strong gut reaction of dislike when I think of this person.",
    "I feel a palpable tension when this person enters the room.",
    "I experience a rush of cautionary anxiety around this person."
]


In [ ]:
# IDT constructs
# Construct 1 should be focal dimension; Construct 2 and 3 should be orbiting dimensions.
Content_reivew_D1 = f'''Here are the construct definitions to use when evaluate items of each dimension:
- Construct 1**Cognitive**: Rational beliefs or expectations about another party’s untrustworthiness, specifying in what respects and under which circumstances distrust may arise.
- Construct 2**Behavioral**: The unwillingness or avoidance of future interactions or risk-taking actions based on these negative expectations and feelings toward the other person.
- Construct 3**Affective**: Negative emotions directed at the distrusted person.'''

Content_reivew_D2 = f'''Here are the construct definitions to use when evaluate items of each dimension:
- Construct 1**Behavioral**: The unwillingness or avoidance of future interactions or risk-taking actions based on these negative expectations and feelings toward the other person.
- Construct 2**Cognitive**: Rational beliefs or expectations about another party’s untrustworthiness, specifying in what respects and under which circumstances distrust may arise.
- Construct 3**Affective**: Negative emotions directed at the distrusted person.'''

Content_reivew_D3 = f'''Here are the construct definitions to use when evaluate items of each dimension:
- Construct 1**Affective**: Negative emotions directed at the distrusted person.
- Construct 2**Cognitive**: Rational beliefs or expectations about another party’s untrustworthiness, specifying in what respects and under which circumstances distrust may arise.
- Construct 3**Behavioral**: The unwillingness or avoidance of future interactions or risk-taking actions based on these negative expectations and feelings toward the other person.'''

In [ ]:
cv_prompt_formatted = []
for i in range(len(items_cognitive)):
# need to revise the definition and target items.
  cv_prompt_formatted.append([
    {"role": "system", "content": f'''**Role assignment**You are a content reviewer.
      **task objective**Your task is to review a item and evaluate how they related to each construct.
      **task instructions**
      - You will be given three construct definitions for the items : {Content_reivew_D2}
      - Assess the degree to which each item matches each of the three given construct definitions using a seven-point response scale, (1 = extremely bad; 2 = very bad; 3 = somewhat bad, 4 = adequate; 5 = somewhat good; 6 = very good; 7 = extremely good).
      - You need to provide a score for each construct, so three scores for each item.
      **Formatting Requirements**
      - Seperate your ratings with coma, start a new line for each item, make sure you rate all 40 items.
      - IMPORTANT: Only provide items, and ratings in your ouput, do not provide any other explanation.
      **Evaluating targets**
      Here is the item that you need to evaluate:
      {items_behavioral[i]}.
      '''},
    {"role": "user", "content": "provide your ratings, do not include the original item in your response, seperate the rating scores by coma"}
  ])

In [ ]:
content_validation_prompt =f'''**Role assignment**You are a content reviewer.
      **task objective**Your task is to review a item and evaluate how they related to each construct.
      **task instructions**
      - You will be given three construct definitions for the items : {Content_reivew_D2}
      - Assess the degree to which each item matches each of the three given construct definitions using a seven-point response scale, (1 = extremely bad; 2 = very bad; 3 = somewhat bad, 4 = adequate; 5 = somewhat good; 6 = very good; 7 = extremely good).
      - You need to provide a score for each construct, so three scores for each item.
      **Formatting Requirements**
      - Seperate your ratings with coma.
      - IMPORTANT: Only provide items, and ratings in your ouput, do not provide any other explanation.
      **Evaluating targets**
      Here is the item that you need to evaluate:
      {items_behavioral}.
      '''

In [ ]:
messages = [
    {"role": "system", "content": content_validation_prompt},
    {"role": "user", "content": "provide your ratings along with each item"}
]

In [ ]:
rating_scores = []
for i in range(len(cv_prompt_formatted)):
  # create gpt generated score
  completion = client.chat.completions.create(
    model='o3-mini',
    # here is your prompt list
    messages=cv_prompt_formatted[i],
#    top_p=0,
#    temperature=0,
    seed = 123
  )
  rating_scores.append(completion.choices[0].message.content)

In [ ]:
rating_scores

['7,1,1',
 '6,1,1',
 '7,1,1',
 '6,2,1',
 '7,1,2',
 '7,2,2',
 '5,2,1',
 '7,2,2',
 '6,1,1',
 '7,1,1',
 '6,2,2',
 '6,2,2',
 '7,1,1',
 '7,1,1',
 '7,1,1',
 '7,2,2',
 '7,1,1',
 '7,1,1',
 '6,2,2',
 '7,2,1',
 '6,1,1',
 '5,2,1',
 '7,1,1',
 '7,1,1',
 '5,1,1',
 '7,1,1',
 '7,1,1',
 '7,1,1',
 '7,2,1',
 '6,2,2',
 '7,1,1',
 '7,2,2',
 '6,1,1',
 '7,1,2',
 '5,1,1',
 '7,1,1',
 '4,2,1',
 '7,1,1',
 '6,1,1',
 '7,2,1']

##Llama 3.3

In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.4 MB/s eta 0:00:00


In [ ]:
import os

from groq import Groq

client = Groq(
    api_key="Your Groq API key",
)


In [ ]:
rating_scores = []
for i in range(len(cv_prompt_formatted)):
  # create gpt generated score
  chat_completion = client.chat.completions.create(
    model='llama-3.3-70b-versatile',
    # here is your prompt list
    messages=cv_prompt_formatted[i],
#    top_p=0,
#    temperature=0,
    seed = 123
  )
  rating_scores.append(chat_completion.choices[0].message.content)

In [ ]:
rating_scores

['6, 2, 1',
 '6, 2, 5',
 '6, 5, 4',
 '6, 2, 3',
 '6, 2, 5',
 '6, 2, 3',
 '6, 2, 3',
 '6, 2, 3',
 '6, 2, 1',
 '6, 5, 4',
 '6, 5, 4',
 '6, 5, 5',
 '6, 2, 5',
 '6, 2, 1',
 '6, 2, 3',
 '6, 2, 3',
 '6, 2, 3',
 '6, 2, 1',
 '6, 2, 3',
 '6, 2, 3',
 '7, 5, 4',
 '6, 2, 3',
 '6, 2, 5',
 '6, 2, 3',
 '6, 2, 5',
 '6, 2, 5',
 '6, 2, 3',
 '6, 2, 3',
 '6, 5, 4',
 '6, 2, 5',
 '6, 2, 3',
 '6, 2, 3',
 '6, 2, 1',
 '6, 2, 3',
 '6, 5, 5',
 '6, 2, 3',
 '6, 2, 3',
 '6, 2, 3',
 '6, 2, 3',
 '6, 2, 5']

##deepseek r1

In [ ]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
# login to your huggingface account
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `NLI` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushi

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [ ]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="cuda")
#model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
import transformers
test_generator = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=4096,
    device_map = "cuda",
    task="text-generation")

Device set to use cuda


In [ ]:
# create a function to generate responses
def get_response(prompt):
  response = test_generator(prompt)
  text = response[0]["generated_text"]
  return text

In [ ]:
response_all = []
for i in range(len(cv_prompt_formatted)):
  response = get_response(cv_prompt_formatted[i])
  response_all.append(response.split("</think>")[-1].strip())
  # create gpt generated score

In [ ]:
response_all

['7,6,4',
 '7,4,3',
 '7,3,3',
 '7,3,2',
 '6,4,4',
 '6,4,3',
 '5,5,3',
 '7,3,2',
 '7,3,1',
 '7,5,4',
 '7,4,3',
 '7,6,5',
 '7,5,3',
 '7,4,3',
 '5,3,3',
 '5,4,3',
 '7,4,3',
 '6,6,4',
 '7,4,3',
 'Cognitive: 5\nBehavioral: 5\nAffective: 4',
 '6,5,4',
 '7,5,3',
 '7,3,1',
 '7,5,3',
 '7,5,3',
 '7,4,3',
 '7,4,3',
 '5,6,3',
 '7,5,4',
 '7,5,3',
 '7,5,3',
 '7,5,3',
 '7,6,4',
 '6,6,4',
 '7,5,3',
 '7,1,1',
 '7,5,5',
 '7,3,2',
 '7,4,4',
 '3,5,6']

In [ ]:
scores_extract = []
for i in range(len(response_all)):
  content_after_think = response_all[i].split("</think>")[-1].strip()
  scores_extract.append(content_after_think)

print(scores_extract)

['6,5,5', '5,4,4', '7,4,3', '7,6,4', '6,5,4', '6,6,6', '7,7,5', '6,5,4', '6,4,5', '7,7,6', '7,6,4', '7,7,5', '7,6,4', '7,6,4', '7,6,4', '7,4,5', '7,7,5', '7,5,4', '7,4,4', '6,4,5', '6,5,4', '7,5,5', '5,5,3', '7,6,4', '7,7,4', '7,5,4', '7,6,5', '6,7,5', '7,5,4', '7,4,5', '7,5,6', '6,7,4', '7,7,5', '7,5,6', '5,5,5', '6,3,5', '7,7,5', '7,6,4', '7,4,4', '7,6,5']


## Gemini

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

genai.configure(api_key='Your Gemini API key')

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')

In [ ]:
cv_prompt_gemini = []
for i in range(len(items_cognitive)):

  cv_prompt_gemini.append([
    f"""**Role assignment**You are a content reviewer.
      **task objective**Your task is to review a item and evaluate how they related to each construct.
      **task instructions**
      - You will be given three construct definitions for the items : {Content_reivew_D3}
      - Assess the degree to which each item matches each of the three given construct definitions using a seven-point response scale, (1 = extremely bad; 2 = very bad; 3 = somewhat bad, 4 = adequate; 5 = somewhat good; 6 = very good; 7 = extremely good).
      - You need to provide a score for each construct, so three scores for each item.
      **Formatting Requirements**
      - Seperate your ratings with coma.
      - IMPORTANT: Only provide items, and ratings in your ouput, do not provide any other explanation.
      **Evaluating targets**
      Here is the item that you need to evaluate:
      {items_affective[i]}.
      """])

In [ ]:
response_all = []
# you may need to adjust the range here due to the request limit you have on Gemini
for i in range(0,40):
  response = model.generate_content(cv_prompt_gemini[i])
  response_all.append(response.text)
  # create gpt generated score

In [ ]:
response_all

['7, 1, 1\n',
 '7, 1, 1\n',
 '7, 2, 2\n',
 '7, 1, 1\n',
 '1, 1, 1\n',
 '7, 2, 2\n',
 '7, 1, 1\n',
 '7, 1, 1\n',
 '7, 1, 1\n',
 '7, 1, 1\n']

#Get embeddings and EFA

model tuning process is in a different file, the fine-tuned model for this construct is "zihoo/all-MiniLM-L6-v2-IDT-multirank"

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the original model and fine-tuned SBERT model
model_raw = SentenceTransformer("all-MiniLM-L6-v2") # original SBERT
model_ft = SentenceTransformer("zihoo/all-MiniLM-L6-v2-IDT-multirank") # Fine-tuned SBERT


In [ ]:
# you can also read the item statements from a csv file
sentences = [
    "I feel uncomfortable around this person.",
    "I worry about future interactions with this person.",
    "I feel tense when I am with this person.",
    "I experience anxiety when interacting with this person.",
    "It is stressful to interact with this person.",
    "This person would engage in damaging and harmful behavior to pursue his/her own interest.",
    "This person would behave in a deceptive and fraudulent way.",
    "Based on past experience, I cannot rely on this person with complete confidence.",
    "I am suspicious of the way this person will act in the future.",
    "This person would use me for his/her own benefits.",
    "I find it necessary to be cautious with this person.",
    "I want to make sure this person has no power or effect on anything I work on.",
    "I will protect myself from being taken advantage of by this person.",
    "I want to distance myself from this person.",
    "I will not count on this person for important things."]

In [ ]:
embeddings_raw = model_raw.encode(sentences, convert_to_numpy=True)
embeddings_ft = model_ft.encode(sentences, convert_to_numpy=True)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
# 4. Compute cosine similarity matrix
similarity_matrix_raw = cosine_similarity(embeddings_raw)
similarity_matrix_ft = cosine_similarity(embeddings_ft)
# similarity_matrix shape: (num_sentences, num_sentences)

# 5. Convert the matrix to a DataFrame for easy saving
df_sim_raw = pd.DataFrame(similarity_matrix_raw)
df_sim_ft = pd.DataFrame(similarity_matrix_ft)
# (Optional) Add row and column labels for clarity
df_sim_raw.index = [f"Sentence_{i}" for i in range(len(sentences))]
df_sim_raw.columns = [f"Sentence_{i}" for i in range(len(sentences))]

df_sim_ft.index = [f"Sentence_{i}" for i in range(len(sentences))]
df_sim_ft.columns = [f"Sentence_{i}" for i in range(len(sentences))]
# 6. Save to CSV (optional)
#df_sim_raw.to_csv("/content/drive/MyDrive/dissertation/Sbert finetune/data/WM/sim_originalitem_raw.csv", index=True)
#df_sim_ft.to_csv("/content/drive/MyDrive/dissertation/Sbert finetune/data/WM/sim_originalitem_ft.csv", index=True)

In [ ]:
!pip install factor_analyzer

In [ ]:
import pandas as pd
from factor_analyzer import FactorAnalyzer

In [ ]:
#scree plot
df_raw = df_sim_raw
fa_test = FactorAnalyzer(n_factors=df_raw.shape[1], rotation=None)
fa_test.fit(df_raw)
ev, v = fa_test.get_eigenvalues()
print("Eigenvalues:\n", ev)

# (Optional) Plot the eigenvalues to visualize the scree plot
import matplotlib.pyplot as plt

plt.scatter(range(1, df_raw.shape[1]+1), ev)
plt.plot(range(1, df_raw.shape[1]+1), ev)
plt.title("Scree Plot")
plt.xlabel("Factor")
plt.ylabel("Eigenvalue")
plt.grid()
plt.show()

In [ ]:
#scree plot
df_ft = df_sim_ft
fa_test = FactorAnalyzer(n_factors=df_ft.shape[1], rotation=None)
fa_test.fit(df_ft)
ev, v = fa_test.get_eigenvalues()
print("Eigenvalues:\n", ev)

# (Optional) Plot the eigenvalues to visualize the scree plot
import matplotlib.pyplot as plt

plt.scatter(range(1, df_ft.shape[1]+1), ev)
plt.plot(range(1, df_ft.shape[1]+1), ev)
plt.title("Scree Plot")
plt.xlabel("Factor")
plt.ylabel("Eigenvalue")
plt.grid()
plt.show()

In [ ]:
# raw model
fa = FactorAnalyzer(is_corr_matrix = True, n_factors=3, rotation="oblimin")
fa.fit(df_raw)

loadings = fa.loadings_.round(2)
print("Factor loadings:\n", loadings)

variance_info = fa.get_factor_variance()
print("Variance explained by each factor:\n", variance_info)

In [ ]:
# fine-tuned model
fa = FactorAnalyzer(n_factors=3, is_corr_matrix=True, rotation="oblimin")
fa.fit(df_mean)

loadings = fa.loadings_.round(2)
print("Factor loadings:\n", loadings)

variance_info = fa.get_factor_variance()
print("Variance explained by each factor:\n", variance_info)